In [ ]:
# “best” support-vector indices for each representation

import numpy as np
import scipy.io
from scipy.linalg import eigh
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# 1) Load QM7, extract 23 eigenvalues per Coulomb matrix => 23 features

def load_qm7_23d(matfile='qm7.mat'):
    """
    Loads the 23 eigenvalues from each 23x23 matrix
    to get an X_23d of shape (N, 23) plus Y of shape (N,).
    """
    print("Loading data from", matfile)
    data = scipy.io.loadmat(matfile)
    C_matrices = data['X']  # shape (N, 23, 23)
    energies   = data['T'].ravel()
    N = C_matrices.shape[0]

    X_23d = []
    for i in range(N):
        M = 0.5*(C_matrices[i] + C_matrices[i].T)
        e_vals, _ = eigh(M)
        e_sorted = np.sort(e_vals)  # shape (23,)
        X_23d.append(e_sorted)
    X_23d = np.array(X_23d)
    print(f"Data loaded: {N} molecules with 23-dimensional features.")
    return X_23d, energies

# 2) Feature scaling, important because of a restricted 0-2pi space in which the datapoints are embbeded in

def scale_features(X, feature_range=(0, 1)):
    """
    Applies min-max scaling to each feature in X so that its values
    lie in the given feature_range.
    """
    print("Scaling features with min-max normalization...")
    X_scaled = np.zeros_like(X)
    a, b = feature_range
    for j in range(X.shape[1]):
        col = X[:, j]
        min_val, max_val = col.min(), col.max()
        if max_val - min_val > 0:
            X_scaled[:, j] = a + (col - min_val) * (b - a) / (max_val - min_val)
        else:
            X_scaled[:, j] = col
    print("Feature scaling complete.")
    return X_scaled

# 3) ZZ-feature map helper

def zz_feature_map_circuit(x, reps=1):
    """
    Builds the ZZ feature map quantum circuit.
    x must be a 23-dimensional input vector.
    """
    dim = len(x)
    if dim != 23:
        raise ValueError("This function is hard-coded for 23-dim input.")
    qc = QuantumCircuit(dim)
    # Initial Hadamard on all qubits
    for qubit in range(dim):
        qc.h(qubit)

    # Applys the pattern for the given number of reps
    for rep in range(reps):
        print(f"  Applying repetition {rep+1}...")
        # Single-qubit rotations
        for i in range(dim):
            angle_i = 2.0 * x[i]
            qc.rz(angle_i, i)
        # Pairwise entangling gates
        for i in range(dim):
            for j in range(i+1, dim):
                angle_ij = 2.0 * x[i] * x[j]
                qc.cx(i, j)
                qc.rz(angle_ij, j)
                qc.cx(i, j)
    return qc

def build_zz_statevector(x, reps=1):
    qc = zz_feature_map_circuit(x, reps=reps)
    return Statevector.from_instruction(qc)

def compute_zz_kernel(X, reps=1):
    """
    Computes the kernel matrix K such that:
       K[i,j] = |<ψ(x_i)|ψ(x_j)>|^2
    Prints progress of both statevector generation and kernel computation.
    """
    N = X.shape[0]
    print(f"\nComputing statevectors for {N} samples:")
    sv_list = []
    for i in range(N):
        sv = build_zz_statevector(X[i], reps=reps)
        print(f"  Processed sample {i+1}/{N}")
        sv_list.append(sv)
    
    K = np.zeros((N, N))
    total_kernels = (N*(N+1)) // 2  # number of unique kernel elements (since symmetric)
    kernel_count = 0
    print("\nComputing the kernel matrix:")
    for i in range(N):
        for j in range(i, N):
            ov = sv_list[i].data.conjugate() @ sv_list[j].data
            K[i, j] = abs(ov)**2
            K[j, i] = K[i, j]
            kernel_count += 1
            # Print progress every 10 kernels (adjust as needed)
            if kernel_count % 10 == 0 or kernel_count == total_kernels:
                print(f"  Processed kernel {kernel_count}/{total_kernels}", end="\r")
    print("\nKernel matrix computation complete.")
    return K

# 4) Main: QSVR on 23-dimensional eigenvalues

def main_manual_zz_feature_map_demo(matfile='qm7.mat', subset_size=50, test_size=0.2, reps=1, random_seed=42):
    # 1) Loads data
    X_all, Y_all = load_qm7_23d(matfile)
    Ntotal = len(X_all)

    # 1.5) Scales the features
    X_all_scaled = scale_features(X_all, feature_range=(0, 1))
    
    # 2) Sub-samples the data
    print(f"\nSub-sampling {subset_size} samples from total {Ntotal} molecules...")
    np.random.seed(random_seed)
    idxs = np.random.choice(Ntotal, size=subset_size, replace=False)
    X_sub = X_all_scaled[idxs]
    Y_sub = Y_all[idxs]

    # 3) Train/Test split
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_sub, Y_sub, test_size=test_size, random_state=random_seed
    )
    print(f"Train size = {X_train.shape[0]}, Test size = {X_test.shape[0]}")

    # 4) Builds kernel on training set
    print(f"\nBuilding manual ZZ feature map with reps={reps} and computing train kernel...")
    K_train = compute_zz_kernel(X_train, reps=reps)

    # 5) Fits SVR with precomputed kernel
    print("\nFitting SVR model with precomputed kernel...")
    svr = SVR(kernel='precomputed', C=1e4, gamma=1e-3, epsilon=0.01)
    svr.fit(K_train, Y_train)
    print("SVR training complete.")

    # 6) Computes test cross-kernel
    print("\nComputing test cross-kernel:")
    N_train = X_train.shape[0]
    N_test  = X_test.shape[0]
    total_test_kernels = N_test * N_train
    test_kernel_count = 0
    sv_test = []
    print(f"Computing statevectors for {N_test} test samples:")
    for i in range(N_test):
        sv = build_zz_statevector(X_test[i], reps=reps)
        print(f"  Processed test sample {i+1}/{N_test}")
        sv_test.append(sv)
    sv_train = []
    print(f"Computing statevectors for {N_train} train samples (for test kernel)...")
    for i in range(N_train):
        sv = build_zz_statevector(X_train[i], reps=reps)
        print(f"  Processed train sample {i+1}/{N_train}")
        sv_train.append(sv)
    
    K_test = np.zeros((N_test, N_train))
    for i in range(N_test):
        for j in range(N_train):
            ov = sv_test[i].data.conjugate() @ sv_train[j].data
            K_test[i, j] = abs(ov)**2
            test_kernel_count += 1
            # Print progress every 5 kernels (adjust if necessary)
            if test_kernel_count % 5 == 0 or test_kernel_count == total_test_kernels:
                print(f"  Processed test kernel {test_kernel_count}/{total_test_kernels}", end="\r")
    print("\nTest kernel computation complete.")

    # 7) Predicts using the SVR model
    print("\nPredicting using SVR model...")
    Y_pred = svr.predict(K_test)

    # 8) Evaluates predictions
    mae = mean_absolute_error(Y_test, Y_pred)
    r2  = r2_score(Y_test, Y_pred)
    print("\nManual ZZ Feature Map QSVM results:")
    print(f"  MAE = {mae:.3f}")
    print(f"  R^2 = {r2:.3f}")

if __name__ == "__main__":
    main_manual_zz_feature_map_demo(
        matfile='qm7.mat',
        subset_size=50,
        test_size=0.2,
        reps=1,
        random_seed=42
    )


In [ ]:
import numpy as np
import scipy.io
from scipy.linalg import eigh
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

BEST_IDS_23 = [
    305, 286, 145, 285, 284, 148, 281, 280, 279, 152,
    143, 277, 276, 157, 158, 275, 161, 274, 273, 270,
    166, 154, 269, 287, 139, 112, 309, 306, 115, 117,
    121, 122, 301, 125, 140, 299, 297, 129, 130, 131,
    292, 290, 135, 137, 138, 298, 268, 169, 263, 249,
    204, 246, 206, 207, 245, 244, 243, 213, 202, 214,
    241, 217, 239, 235, 221, 232, 231, 224, 230, 242,
    251, 252, 198, 262, 173, 175, 176, 177, 178, 179,
    180, 261, 260, 259, 185, 186, 187, 188, 189, 257,
    193, 194, 196, 256, 111, 311, 227, 107,  38,  39,
    40,  41, 361,  45,  48,  49, 366,  50,  52, 109,
     54,  56, 354,  61, 352, 351,  51,  64, 368, 373,
    397,   5,   6, 390,   9, 384, 383,  15,  29, 382,
    379,  21, 377, 375, 374,  25,  26,  27,  18, 350,
     53, 349,  85,  87,  88,  66,  91,  92,  93,  95,
     96, 325, 321, 101, 103, 105, 106, 331,  83,  89,
    334, 348, 345,  82,  70,  74, 344, 341,  71,  79,
    336, 303, 102,  34, 132,  46, 191, 160, 389, 392,
     86, 199, 391, 313, 182, 174, 155, 381, 209, 362,
    304,  14, 386, 317,  31, 226, 367,  72, 358, 378,
    219,  73,   0, 393, 104, 201, 267,   7, 237,   2,
    387, 134, 114, 324,  37, 222,  81, 288, 123, 310,
    372, 212,   1,  65, 332, 225, 156, 228,  17, 172,
    283, 167, 124,  62,  80, 380, 144, 162, 234, 320,
    216, 371, 388, 147, 338, 218, 238,  12,  20,  67,
    370, 150, 343, 210, 159, 319, 340, 247, 183,  43,
    363, 240, 289, 385, 127, 314, 168, 295,  24, 335,
    369, 220, 116, 236, 328, 339, 133, 398, 399, 357,
    100,   4, 322, 315, 360, 163,  23, 356, 265, 266,
    192, 396,  13, 151,  33,  16, 253,  84, 141, 171
]

# 1) Load QM7, extract 23 eigenvalues per Coulomb matrix => 23 features
def load_qm7_23d(matfile='qm7.mat'):
    """
    Loads the 23 eigenvalues from each 23x23 matrix
    to get an X_23d of shape (N, 23) plus Y of shape (N,).
    """
    print("Loading data from", matfile)
    data = scipy.io.loadmat(matfile)
    C_matrices = data['X']  # shape (N, 23, 23)
    energies   = data['T'].ravel()
    N = C_matrices.shape[0]

    X_23d = []
    for i in range(N):
        M = 0.5*(C_matrices[i] + C_matrices[i].T)
        e_vals, _ = eigh(M)
        e_sorted = np.sort(e_vals)  # shape (23,)
        X_23d.append(e_sorted)
    X_23d = np.array(X_23d)
    print(f"Data loaded: {N} molecules with 23-dimensional features.")
    return X_23d, energies

# 2) Feature scaling, important because of a restricted 0-2pi space in which the datapoints are embedded in
def scale_features(X, feature_range=(0, 1)):
    """
    Applies min-max scaling to each feature in X so that its values
    lie in the given feature_range.
    """
    print("Scaling features with min-max normalization...")
    X_scaled = np.zeros_like(X)
    a, b = feature_range
    for j in range(X.shape[1]):
        col = X[:, j]
        min_val, max_val = col.min(), col.max()
        if max_val - min_val > 0:
            X_scaled[:, j] = a + (col - min_val) * (b - a) / (max_val - min_val)
        else:
            X_scaled[:, j] = col
    print("Feature scaling complete.")
    return X_scaled

# 3) ZZ-feature map helper
def zz_feature_map_circuit(x, reps=1):
    """
    Builds the ZZ feature map quantum circuit.
    x must be a 23-dimensional input vector.
    """
    dim = len(x)
    if dim != 23:
        raise ValueError("This function is hard-coded for 23-dim input.")
    qc = QuantumCircuit(dim)
    # Initial Hadamard on all qubits
    for qubit in range(dim):
        qc.h(qubit)

    # Applies the pattern for the given number of reps
    for rep in range(reps):
        print(f"  Applying repetition {rep+1}...")
        # Single-qubit rotations
        for i in range(dim):
            angle_i = 2.0 * x[i]
            qc.rz(angle_i, i)
        # Pairwise entangling gates
        for i in range(dim):
            for j in range(i+1, dim):
                angle_ij = 2.0 * x[i] * x[j]
                qc.cx(i, j)
                qc.rz(angle_ij, j)
                qc.cx(i, j)
    return qc

def build_zz_statevector(x, reps=1):
    qc = zz_feature_map_circuit(x, reps=reps)
    return Statevector.from_instruction(qc)

def compute_zz_kernel(X, reps=1):
    """
    Computes the kernel matrix K such that:
       K[i,j] = |<ψ(x_i)|ψ(x_j)>|^2
    Prints progress of both statevector generation and kernel computation.
    """
    N = X.shape[0]
    print(f"\nComputing statevectors for {N} samples:")
    sv_list = []
    for i in range(N):
        sv = build_zz_statevector(X[i], reps=reps)
        print(f"  Processed sample {i+1}/{N}")
        sv_list.append(sv)
    
    K = np.zeros((N, N))
    total_kernels = (N*(N+1)) // 2
    kernel_count = 0
    print("\nComputing the kernel matrix:")
    for i in range(N):
        for j in range(i, N):
            ov = sv_list[i].data.conjugate() @ sv_list[j].data
            K[i, j] = abs(ov)**2
            K[j, i] = K[i, j]
            kernel_count += 1
            if kernel_count % 10 == 0 or kernel_count == total_kernels:
                print(f"  Processed kernel {kernel_count}/{total_kernels}", end="\r")
    print("\nKernel matrix computation complete.")
    return K

# 4) Main: QSVR on 23-dimensional eigenvalues
def main_manual_zz_feature_map_demo(
    matfile='qm7.mat',
    subset_size=50,
    test_size=0.2,
    reps=1,
    random_seed=42
):
    # 1) Loads data
    X_all, Y_all = load_qm7_23d(matfile)
    Ntotal = len(X_all)

    # 1.5) Scales the features
    X_all_scaled = scale_features(X_all, feature_range=(0, 1))
    
    # 2) Sub-samples the data
    print(f"\nSub-sampling {subset_size} samples from total {Ntotal} molecules...")
    # ← replaced random with top IDs:
    idxs = BEST_IDS_23[:subset_size]
    X_sub = X_all_scaled[idxs]
    Y_sub = Y_all[idxs]

    # 3) Train/Test split
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_sub, Y_sub, test_size=test_size, random_state=random_seed
    )
    print(f"Train size = {X_train.shape[0]}, Test size = {X_test.shape[0]}")

    # 4) Builds kernel on training set
    print(f"\nBuilding manual ZZ feature map with reps={reps} and computing train kernel...")
    K_train = compute_zz_kernel(X_train, reps=reps)

    # 5) Fits SVR with precomputed kernel
    print("\nFitting SVR model with precomputed kernel...")
    svr = SVR(kernel='precomputed', C=1e4, gamma=1e-3, epsilon=0.01)
    svr.fit(K_train, Y_train)
    print("SVR training complete.")

    # 6) Computes test cross-kernel
    print("\nComputing test cross-kernel:")
    N_train = X_train.shape[0]
    N_test  = X_test.shape[0]
    total_test_kernels = N_test * N_train
    test_kernel_count = 0
    sv_test = []
    print(f"Computing statevectors for {N_test} test samples:")
    for i in range(N_test):
        sv = build_zz_statevector(X_test[i], reps=reps)
        print(f"  Processed test sample {i+1}/{N_test}")
        sv_test.append(sv)
    sv_train = []
    print(f"Computing statevectors for {N_train} train samples (for test kernel)...")
    for i in range(N_train):
        sv = build_zz_statevector(X_train[i], reps=reps)
        print(f"  Processed train sample {i+1}/{N_train}")
        sv_train.append(sv)
    
    K_test = np.zeros((N_test, N_train))
    for i in range(N_test):
        for j in range(N_train):
            ov = sv_test[i].data.conjugate() @ sv_train[j].data
            K_test[i, j] = abs(ov)**2
            test_kernel_count += 1
            if test_kernel_count % 5 == 0 or test_kernel_count == total_test_kernels:
                print(f"  Processed test kernel {test_kernel_count}/{total_test_kernels}", end="\r")
    print("\nTest kernel computation complete.")

    # 7) Predicts using the SVR model
    print("\nPredicting using SVR model...")
    Y_pred = svr.predict(K_test)

    # 8) Evaluates predictions
    mae = mean_absolute_error(Y_test, Y_pred)
    r2  = r2_score(Y_test, Y_pred)
    print("\nManual ZZ Feature Map QSVM results:")
    print(f"  MAE = {mae:.3f}")
    print(f"  R^2 = {r2:.3f}")

if __name__ == "__main__":
    main_manual_zz_feature_map_demo(
        matfile='qm7.mat',
        subset_size=300,  #works up to 300
        test_size=0.2,
        reps=1,
        random_seed=42
    )
